In [0]:
from pyspark.sql.functions import col

# Read CSV with schema + badRecordsPath
schema = "Year INT, Total_Demand INT, _corrupt_record STRING"

df = spark.read.format("csv") \
    .schema(schema) \
    .option("enforceSchema", True)\
    .option("header", True) \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_corrupt_record") \
    .load("abfss://gold@rmpyru.dfs.core.windows.net/GoldData.csv")
display(df)

# Separate clean and corrupt records

clean_df = df.filter(col("_corrupt_record").isNull()).drop("_corrupt_record")
bad_df = df.filter(col("_corrupt_record").isNotNull())

display(clean_df)
display(bad_df)

# Write clean data
clean_df.write.mode("overwrite").format("parquet").save('abfss://gold@rmpyru.dfs.core.windows.net/cleanrecords')

# Write bad records
bad_df.write.mode("overwrite").format("parquet").save('abfss://gold@rmpyru.dfs.core.windows.net/quarantined_rows/')



In [0]:
df = spark.read.format("parquet") \
    .option("header", True) \
    .load("abfss://gold@rmpyru.dfs.core.windows.net/quarantined_rows/")
display(df)